## Importações

<h4>Bibliotecas que foram utilizadas durante o desenvolvimento do código e seus objetivos:</h4>

- Sklearn : 
    
    Para avaliar o desempenho dos modelos utilizando a técnica do Cross Validation (model_selection)

    Encontrar o melhor conjunto de parametros para o modelo de acordo com as informações apresentadas (model_selection)

    Para captar os modelos (naive_bayes, neighbors)

- Pandas :
    
    Disponibiliza ferramentas para manipular DataFrames, como por exemplo a forma que são apresentadas as bases

- Pickle:
    
    É usado para extrair as bases do arquivo "bases.pk1" e para armazenar o modelo escolhido em um arquivo .pk1

- Numpy

    No contexto deste notebook, ele auxilia nos intervalos de números que são apresentados nos parametros dos modelos.

In [1]:
# Pandas
import pandas as pd

# Pickle
import pickle

# Numpy
import numpy as np

# Sklearn
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

import math

# Analisando Desempenho de Modelos

## Teste de desempenho entre os modelos e as bases

### <h1 style='color:red'>FALTA TESTAR NORMALIZAÇÃO DE BASES VARIADAS</h1>

### Seleção dos modelos

In [2]:
parametros = {}

# Naive Bayes
parametros['naive_bayes'] = []

for i in range(1, 11):
     parametros['naive_bayes'].append({
        'model__var_smoothing': np.logspace(0,-9, num=100),
        'scaler': [StandardScaler()],
        'SelectKBest__k': [i],
        'pca__n_components': [n for n in range(1, i+1)],
        'model' : [GaussianNB()]
    })

# Arvore - Tree
parametros['tree'] = []

for i in range(1, 11):
    parametros['tree'].append({
        'model__criterion' : ['gini', 'entropy'],
        'model__splitter' : ['best', 'random'],
        'model__max_depth' : [None, 2, 4, 6, 8, 10, 12],
        'model__min_samples_split' : [2, 5, 10],
        'model__min_samples_leaf' : [2, 5, 10],
        'model__max_features' : [None, 'sqrt', 'log2'],
        'scaler': [StandardScaler()],
        'SelectKBest__k': [i],
        'pca__n_components': [n for n in range(1, i + 1)],
        'model' : [tree.DecisionTreeClassifier()]
    })

# KNN 
parametros['knn'] = []

for i in range(1, 11):
    parametros['knn'].append({
        'model__n_neighbors' : np.arange(3, 15),
        'model__weights' : ['uniform', 'distance'],
        'model__algorithm' : ['auto', 'kd_tree'],
        'model__leaf_size' : np.arange(10, 30, 5),
        'model__p' : [1, 2, 3],
        'model__metric' : ['minkowski', 'manhattan'],
        'model__n_jobs' : [1, -1],
        'scaler': [StandardScaler()],
        'SelectKBest__k': [i],
        'pca__n_components': [n for n in range(1, i+1)],
        'model' : [KNeighborsClassifier()]
    })

### Seleção das Bases

In [3]:
# Extraindo as bases de dados limpas que foram armazenadas no arquivo .pk1
bases_armazenadas = pickle.load(open('./bases/bases.pk1','rb'))

values_train = {}
values_val = {}
values_test = {}

for key, value in bases_armazenadas.items():
    atributo = value.iloc[:, :-1]
    resposta = value.iloc[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(atributo, resposta, test_size=0.2, random_state=42)
    values_train[key] = [X_train, y_train]

    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
    values_val[key] = [X_val, y_val]
    values_test[key] = [X_test, y_test]

### Pipeline

In [4]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('SelectKBest', SelectKBest(score_func=f_classif)),
    ('pca', PCA()),
    ('model', tree.DecisionTreeClassifier())
])

### Encontrando as melhores combinações

In [5]:
modelos_f1 = {}
infos = {}

for i in parametros.keys():
    grid = GridSearchCV(estimator=pipeline, param_grid=parametros[i], scoring='f1', cv=5)
    f1 = {}
    overfit = {}

    for x in values_train.keys():
        grid.fit(values_train[x][0], values_train[x][1])
        print('foi')
        chave = x+'_'+i
        resultados = grid.cv_results_
        best_index = grid.best_index_

        infos[chave] = [
            grid.best_estimator_,   
            grid.best_params_,
            resultados['std_test_score'][best_index]
            ]
        
        media_f1 = resultados['mean_test_score'][best_index]

        if media_f1 < 0.98:
            f1[chave] = media_f1
        else:
            overfit[chave] = media_f1
    
    modelos_f1[i] = [f1, overfit]

foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi
foi


### Gráfico de desempenho das bases

##### Naive Bayes

In [ ]:
fig_bayes = go.Figure()

x = list(modelos_f1['naive_bayes'][0].keys())
y = list(modelos_f1['naive_bayes'][0].values())

y_formatted = [round(value, 3) for value in y]

fig_bayes.add_bar(x=x, y=y, text=y_formatted, textposition='auto', textangle=-360)

fig_bayes.update_layout(
    title='Performace das bases com o Naive Bayes',
    xaxis_title='Bases',
    yaxis_title='F1 Score'
)

fig_bayes.show()

##### Tree

In [ ]:
fig_tree = go.Figure()

x = list(modelos_f1['tree'][0].keys())
y = list(modelos_f1['tree'][0].values())

y_formatted = [round(value, 3) for value in y]

fig_tree.add_bar(x=x, y=y, text=y_formatted, textposition='auto', textangle=-360)

fig_tree.update_layout(
    title='Performace das bases com a Tree',
    xaxis_title='Bases',
    yaxis_title='F1 Score'
)

fig_tree.show()

##### KNN

In [ ]:
fig_knn = go.Figure()

x = list(modelos_f1['knn'][0].keys())
y = list(modelos_f1                                                                                                                                                                                                                     ['knn'][0].values())

y_formatted = [round(value, 3) for value in y]

fig_knn.add_bar(x=x, y=y, text=y_formatted, textposition='auto', textangle=-360)

fig_knn.update_layout(
    title='Performace das bases com a KNN',
    xaxis_title='Bases',
    yaxis_title='F1 Score'
)

fig_knn.show()

#### Proximos passos:

- Testar os dois melhores modelos com o val da sua base
- Teste AB
- Teste

## Teste A/B

### Tree

Primeiro é preciso conferir se estão sendo usados parametros diferentes em cada um

In [ ]:
# print(f'Modelos:\n\n Pa - {infos['df_kmeans_smote_tree'][0]}\n Pb - {infos['df_svm_smote_tree'][0]}')

Modelos:

 Pa - DecisionTreeClassifier(criterion='entropy', max_features='log2',
                       splitter='random')
 Pb - DecisionTreeClassifier(splitter='random')


Objetivo - Entender se a base df_kmeans_smote_tree com o modelo que melhor interagiu com ela, apresenta um f1 score significativamente maior que o f1 score apresentado pela base df_svm_smote_tree em conjunto com o modelo que melhor interagiu com ela. Para ao final, descobrir qual das duas é melhor para ser utilizada em comparação com as outras combinações de base e modelo.

Pa = df_kmeans_smote_tree <br>
Pb = df_svm_smote_tree

Desta forma as hipoteses seria:
H0 = Pa < Pb
H1 = Pa > Pb

Significancia: <br>
α = 5% = 0,05

In [ ]:
# pa = acuracias_modelo['tree'][0]['df_kmeans_smote_tree']
# pb = acuracias_modelo['tree'][0]['df_svm_smote_tree']

# p_estimado = ((pa*5)+(pb*5))/(5+5)

# z = (pa-pb)/(math.sqrt(p_estimado*(1-p_estimado)*((1/5)+(1/5))))

# print(z)

# if z >= 1.645:
#     print('A hipotese 0 foi rejeitada, logo a grandes indicios de o f1 score apresentado pelo df_kmeans_smote_knn ser significativamente maior que a metrica apresentada pelo df_svm_smote_tree.')
# else:
#     print('A hipotese 0 não foi rejeitada, logo não tem indicios de o f1 score do df_kmeans_smote_knn ser significativamente maior.')

-0.0014363677826791045
A hipotese 0 não foi rejeitada, logo não tem indicios de o f1 score do df_kmeans_smote_knn ser significativamente maior.


### KNN

Primeiro é preciso conferir se estão sendo usados parametros diferentes em cada um

In [ ]:
# print(f'Modelos:\n\n Pa - {infos['df_kmeans_smote_knn'][0]}\n Pb - {infos['df_boderline_smote_knn'][0]}')

Modelos:

 Pa - KNeighborsClassifier(leaf_size=10, n_jobs=1, n_neighbors=4, weights='distance')
 Pb - KNeighborsClassifier(leaf_size=10, n_jobs=1, n_neighbors=4, p=1,
                     weights='distance')


Objetivo - Entender se a base df_kmeans_smote_knn com o modelo que melhor interagiu com ela, apresenta um f1 score significativamente maior que o f1 score apresentado pela base df_boderline_smote_knn em conjunto com o modelo que melhor interagiu com ela. Para ao final, descobrir qual das duas é melhor para ser utilizada em comparação com as outras combinações de base e modelo.

Pa = df_kmeans_smote_knn <br>
Pb = df_boderline_smote_knn

Desta forma as hipoteses seria:<br>
H0 = Pa < Pb<br>
H1 = Pa > Pb

Significancia: <br>
α = 5% = 0,05

In [ ]:
# pa = acuracias_modelo['knn'][0]['df_kmeans_smote_knn']
# pb = acuracias_modelo['knn'][0]['df_boderline_smote_knn']

# p_estimado = ((pa*5)+(pb*5))/(5+5)

# z = (pa-pb)/(math.sqrt(p_estimado*(1-p_estimado)*((1/5)+(1/5))))

# print(z)

# if z >= 1.645:
#     print('A hipotese 0 foi rejeitada, logo a grandes indicios de o f1 score apresentado pelo df_kmeans_smote_knn ser significativamente maior que a metrica apresentada pelo df_boderline_smote_knn.')
# else:
#     print('A hipotese 0 não foi rejeitada, logo não tem indicios de o f1 score do df_kmeans_smote_knn ser significativamente maior.')

0.22727501492523064
A hipotese 0 não foi rejeitada, logo não tem indicios de o f1 score do df_kmeans_smote_knn ser significativamente maior.
